#Загружаем transactions, clickstream и train matching

In [ ]:
!wget https://storage.yandexcloud.net/datasouls-ods/materials/acfacf11/train_matching.csv

!wget https://storage.yandexcloud.net/datasouls-ods/materials/b949c04c/mcc_codes.csv
!wget https://storage.yandexcloud.net/datasouls-ods/materials/705abbab/click_categories.csv
!wget https://storage.yandexcloud.net/datasouls-ods/materials/e33f2201/currency_rk.csv

!wget https://storage.yandexcloud.net/datasouls-ods/materials/0433a4ca/transactions.zip
!unzip transactions.zip > /dev/null

!wget https://storage.yandexcloud.net/datasouls-ods/materials/0554f0cf/clickstream.zip
!unzip clickstream.zip > /dev/null

--2022-05-18 10:54:39--  https://storage.yandexcloud.net/datasouls-ods/materials/acfacf11/train_matching.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1070145 (1.0M) [text/csv]
Saving to: ‘train_matching.csv’

train_matching.csv  100%[===================>]   1.02M  1.08MB/s    in 0.9s    

2022-05-18 10:54:41 (1.08 MB/s) - ‘train_matching.csv’ saved [1070145/1070145]

--2022-05-18 10:54:41--  https://storage.yandexcloud.net/datasouls-ods/materials/b949c04c/mcc_codes.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200290 (196K) [text/csv]
Saving to: ‘mcc_codes.csv’

mcc_codes.csv     

In [ ]:
!pip install dask[dataframe] --upgrade

     |████████████████████████████████| 136 kB 5.2 MB/s 


#Добавляем фичи

In [ ]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import dask.dataframe as dd
import dask.dataframe.reshape


In [ ]:
transactions = pd.read_csv('./transactions.csv').fillna(0)
clickstream = dd.read_csv('./clickstream.csv')
train_matching = pd.read_csv('./train_matching.csv')
mcc_codes = pd.read_csv('./mcc_codes.csv')
click_categories = pd.read_csv('./click_categories.csv')
currency_rk = pd.read_csv('./currency_rk.csv')
min_date = datetime.fromisoformat('2020-01-01 00:00:01').toordinal() #точка отсчета количества недель 
transactions['transaction_dttm'] = [datetime.fromisoformat(date_dttm).toordinal() - min_date for date_dttm in transactions["transaction_dttm"]]
clickstream = clickstream.merge(dd.from_pandas(pd.DataFrame({"timestamp_day":[datetime.fromisoformat(date_timestamp).toordinal() - min_date for date_timestamp in clickstream["timestamp"]]}, dtype='int16'), npartitions=1), how='left')
clickstream = clickstream.merge(dd.from_pandas(pd.DataFrame({"uid":[uid for uid in clickstream["new_uid"]]}, dtype='int32'), npartitions=1), how='left')

In [ ]:
aggfunc = ['sum', 'mean', 'count']
flag = True

for currency_code in currency_rk['currency_rk']:                                                #считываем индекс валюты
    currency = currency_rk["Name"].loc[currency_rk["currency_rk"]==currency_code].values[0] #название валюты

    bankclient_currency = transactions.loc[transactions["currency_rk"] == currency_code].pivot_table(index = 'user_id', 
                         values='currency_rk',
                         columns='mcc_code',
                         aggfunc='count').fillna(0)
    bankclient_currency.columns = [f'{currency}-{i}' for i in bankclient_currency]
    if flag:
        flag = False
        bankclient_embed = bankclient_currency
    else:
        bankclient_embed = bankclient_embed.merge(bankclient_currency, how='left', right_index=True, left_index=True)

    for func in aggfunc:
        bankclient = transactions.loc[(transactions["currency_rk"] == currency_code) 
                                    & (transactions["transaction_amt"] >= 0)].pivot_table(index = 'user_id', 
                            values = 'transaction_amt',
                            columns = 'mcc_code',
                            aggfunc = func).fillna(0)
        bankclient.columns = [f'{func}-{i}-{currency}-recharge' for i in bankclient]        
        bankclient_embed = bankclient_embed.merge(bankclient, how='left', right_index=True, left_index=True)

        bankclient = transactions.loc[(transactions["currency_rk"] == currency_code) 
                                    & (transactions["transaction_amt"] <= 0)].pivot_table(index = 'user_id', 
                            values = 'transaction_amt',
                            columns = 'mcc_code',
                            aggfunc = func).fillna(0)
        bankclient.columns = [f'{func}-{i}-{currency}-write-off' for i in bankclient]        
        bankclient_embed = bankclient_embed.merge(bankclient, how='left', right_index=True, left_index=True)

    bankclient_date = transactions.loc[transactions["currency_rk"] == currency_code].pivot_table(index = 'user_id',  #количество оплат на сайте
                            values = 'transaction_dttm',
                            columns = 'mcc_code',
                            aggfunc = 'count').fillna(0)
    bankclient_date.columns = [f'mcc_code_{i}-{currency}' for i in bankclient_date]
    bankclient_embed = bankclient_embed.merge(bankclient_date, how='left', right_index=True, left_index=True)

    bankclient_date = transactions.loc[transactions["currency_rk"] == currency_code].pivot_table(index = 'user_id',  #количество оплат в течение дня
                            values = 'mcc_code',
                            columns = 'transaction_dttm',
                            aggfunc = 'count').fillna(0)
    bankclient_date.columns = [f'day_{i}-{currency}' for i in bankclient_date]
    bankclient_embed = bankclient_embed.merge(bankclient_date, how='left', right_index=True, left_index=True)
bankclient_embed = bankclient_embed.fillna(0)

In [ ]:
bankclient_embed.to_csv('data/bankclient_embed')

In [ ]:
clickstream = clickstream.categorize(columns={'cat_id'})
clickstream = clickstream.categorize(columns={'timestamp_day'})
clickstream = clickstream.categorize(columns={'uid'})

clickstream_embed_count = clickstream.pivot_table(index='user_id', 
                         values='timestamp_day',
                         columns='cat_id',
                         aggfunc='count').fillna(0)
clickstream_embed_count.columns = [f'count-cat_id-{i}' for i in clickstream_embed_count]
clickstream_embed = clickstream_embed_count

clickstream_embed_day = clickstream.pivot_table(index='user_id', 
                         values='cat_id',
                         columns='timestamp_day',
                         aggfunc='count').fillna(0)
clickstream_embed_day.columns = [f'count-day-{i}' for i in clickstream_embed_day]
clickstream_embed = clickstream_embed.merge(clickstream_embed_day, how='left')

clickstream_embed_uid = clickstream.pivot_table(index='user_id', 
                         values='timestamp_day',
                         columns='uid',
                         aggfunc='count').fillna(0)
clickstream_embed_uid.columns = [f'count-uid-{i}' for i in clickstream_embed_uid]
clickstream_embed = clickstream_embed.merge(clickstream_embed_uid, how='left').fillna(0)

In [ ]:
clickstream_embed.to_csv('data/clickstream_embed*')

#Генерируем негативные ....

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# clickstream_embed = pd.read_csv('/content/drive/MyDrive/data/clickstream_embed0.csv', nrows=15000).set_index('user_id')
# bankclient_embed = pd.read_csv('/content/drive/MyDrive/data/bankclient_embed.csv', nrows=15000).set_index('user_id')

In [ ]:
def gen_random_for_negative(x, k, list_of_uniq): 
    while True:
        final_list = np.random.choice(list_of_uniq, size=k, replace=False)
        if x not in final_list:
            return final_list

In [ ]:
list_of_uniq_rtk = train_matching['rtk'].unique()
list_of_uniq_bank = train_matching['bank'].unique()
train_matching['negatives'] = train_matching['rtk'].apply(lambda x: gen_random_for_negative(x, 3, list_of_uniq_rtk))

train_matching['target'] = 1
pos_part = train_matching[['bank', 'rtk', 'target']]
neg_part = train_matching[['bank', 'negatives']].explode('negatives')
neg_part['target'] = 0
neg_part.columns = pos_part.columns

full_dataset = pd.concat([pos_part, neg_part]).sort_values(by='bank')

In [ ]:
del transactions, bankclient_currency, bankclient, bankclient_date, clickstream

In [ ]:
gc.collect()

296

In [ ]:
full_dataset = full_dataset.merge(bankclient_embed, how='inner', left_on='bank', right_index=True).merge(clickstream_embed, how='inner', left_on='rtk', right_index=True).fillna(0)

In [ ]:
full_dataset

,bank,rtk,target,RUR--1,RUR-742,RUR-763,RUR-780,RUR-1520,RUR-1711,RUR-1731,...,count-uid-396,count-uid-1642215,count-uid-148279,count-uid-1427153,count-uid-1707485,count-uid-1074426,count-uid-516519,count-uid-563203,count-uid-1811898,count-uid-1090497
3418,000932580e404dafbecd5916d4640938,297f2ac66c8049e79d07012130a650de,0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6189,576b19060c2944d98c958b0209dbc72d,297f2ac66c8049e79d07012130a650de,0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11892,6a365d3e3b3641c89171b56a284a3324,297f2ac66c8049e79d07012130a650de,0,89.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14915,7955e2bde9384c4bb496b1fd948725e5,297f2ac66c8049e79d07012130a650de,0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538,a39d17e13c43428a91c88b368e54bda1,297f2ac66c8049e79d07012130a650de,1,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,a95e52c1409f48a2a94b54cb41af6b90,afafa25864164fd29bf9b93ec154cd00,1,22.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,300.0,485.0,0.0,0.0,0.0,0.0,0.0,0.0
1938,a96daaf6aa704e1abea5fcbbc5cf9bab,75165697b981443fbee5f95162e11d12,0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7251,a98b923aee4848f9a5e6712f9ec67f13,8866922edd5145bca23cd2a0584169be,1,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
528,a999fa36e96e4ea69db22cf43b0a7d85,b53dbf6914bc44cf8a363e0c6dd03e06,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,134.0,4311.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
list_of_uniq_bank = list_of_uniq_bank.tolist()
np.random.shuffle(list_of_uniq_bank)
train_size = int(0.8 * len(list_of_uniq_bank))

In [ ]:
train = full_dataset[full_dataset['bank'].isin(list_of_uniq_bank[:train_size])]
test = full_dataset[full_dataset['bank'].isin(list_of_uniq_bank[train_size:])]

In [ ]:
test.to_csv('data/test.csv', index=False)
train.to_csv('data/train.csv', index=False)